# Crossover and Mutation Operator Exploration and Tests
This notebook contains our approach to explore crossover and mutation operators to be later included in the library.

In [1]:
import sys
sys.path.append('..')

In [ ]:
import random
from copy import deepcopy

from library.custom.custom_solutions import WSOSolution

# CROSSOVER OPERATORS

## Tag and Untag Helper Functions

In [11]:
# 'tagging' function that gives each table instance a unique position
# imagine this as a seat number at a table

def tag_solution(solution_repr):
    counts = {}
    tagged_soln = []
    for table in solution_repr:
        if table not in counts:
            counts[table] = 0 # setting first instance of each table to 0
        tagged_soln.append((table, counts[table]))
        counts[table] += 1
    return tagged_soln


# 'untagging' function to return solution to original format
def untag_solution(tagged_solution):
    return [table for table, _ in tagged_solution]

## Cycle Crossover

In [12]:
# from lab week 8, some adaptations to make suitable for grouping
def cycle_crossover(parent1_repr, parent2_repr):

    '''
    Defintion
        Cycle Crossover preserves the position of elements by identifying a cycle
        of indices where the values from each parent will be inherited by each offspring.
        The remaining indices are filled with values from the other parent, maintaining valid permutations.

    Parameters
        parent1_repr: Parent 1 representataion (list)
        parent2_repr: Parent 2 representation (list)

    Returns
        Pair of offspring with same type as parents
    '''

    tagged_p1 = tag_solution(parent1_repr)
    tagged_p2 = tag_solution(parent2_repr)

    initial_random_idx = random.randint(0, len(parent1_repr) - 1)

    # Initialize the cycle with the starting index
    cycle_idxs = [initial_random_idx]
    current_cycle_idx = initial_random_idx

    # Traverse the cycle by following the mapping from parent2 to parent1
    while True:
        value_parent2 = tagged_p2[current_cycle_idx]
        next_cycle_idx = tagged_p1.index(value_parent2)

        # break when we close the cycle
        if next_cycle_idx == initial_random_idx:
            break

        cycle_idxs.append(next_cycle_idx)
        current_cycle_idx = next_cycle_idx


    # building offspring
    tagged_offspring1 = []
    tagged_offspring2 = []

    for idx in range(len(parent1_repr)):
        if idx in cycle_idxs:
            # Keep values from parent1 in offspring1 in the cycle indexes
            tagged_offspring1.append(tagged_p1[idx])
            # Keep values from parent2 in offspring2 in the cycle indexes
            tagged_offspring2.append(tagged_p2[idx])
        else:
            # Swap elements from parents in non-cycle indexes
            tagged_offspring1.append(tagged_p2[idx])
            tagged_offspring2.append(tagged_p1[idx])

    # untag offspring to bring back to original format
    offspring1_repr = untag_solution(tagged_offspring1)
    offspring2_repr = untag_solution(tagged_offspring2)

    return offspring1_repr, offspring2_repr

In [13]:
# testing cycle crossover with toy values
parent1 = [1, 2, 1, 3, 1, 3, 3, 2, 2]
parent2 = [3, 2, 1, 1, 1, 2, 2, 3, 3]

print('Parent 1:', parent1)
print('Parent 2:', parent2)

print('Tagged parent 1:', tag_solution(solution_repr=parent1))
print('Tagged parent 2:', tag_solution(solution_repr=parent2))

offspring1, offspring2 = cycle_crossover(parent1_repr=parent1, parent2_repr=parent2)

print('Offspring 1:', offspring1)
print('Offspring 2:', offspring2)

Parent 1: [1, 2, 1, 3, 1, 3, 3, 2, 2]
Parent 2: [3, 2, 1, 1, 1, 2, 2, 3, 3]
Tagged parent 1: [(1, 0), (2, 0), (1, 1), (3, 0), (1, 2), (3, 1), (3, 2), (2, 1), (2, 2)]
Tagged parent 2: [(3, 0), (2, 0), (1, 0), (1, 1), (1, 2), (2, 1), (2, 2), (3, 1), (3, 2)]
Offspring 1: [1, 2, 1, 3, 1, 2, 2, 3, 3]
Offspring 2: [3, 2, 1, 1, 1, 3, 3, 2, 2]


In [14]:
# testing on a real solution
parent1 = WSOSolution().repr
parent2 = WSOSolution().repr

print('Parent 1:', parent1)
print('Parent 2:', parent2)

offspring1_repr, offsrping2_repr = cycle_crossover(parent1_repr=parent1, parent2_repr=parent2)

print('Offspring 1 Representation:', offspring1_repr)
print('Offspring 2 Representation:', offsrping2_repr)

Parent 1: [7, 1, 1, 4, 5, 0, 4, 3, 0, 7, 6, 4, 2, 0, 6, 0, 7, 5, 5, 7, 2, 0, 1, 5, 2, 7, 4, 3, 6, 7, 5, 6, 3, 1, 6, 3, 6, 0, 4, 6, 2, 5, 1, 2, 2, 7, 6, 5, 4, 4, 1, 0, 1, 4, 5, 7, 0, 2, 3, 2, 3, 3, 1, 3]
Parent 2: [4, 2, 2, 5, 0, 7, 7, 2, 5, 4, 0, 6, 3, 2, 7, 7, 3, 1, 5, 5, 6, 1, 4, 6, 7, 1, 1, 4, 0, 0, 1, 4, 0, 5, 4, 6, 2, 6, 0, 7, 3, 6, 7, 2, 4, 1, 0, 3, 4, 1, 7, 1, 2, 6, 5, 0, 3, 5, 3, 6, 3, 3, 2, 5]
Offspring 1 Representation: [4, 2, 1, 5, 0, 7, 7, 2, 5, 4, 0, 6, 3, 0, 6, 0, 7, 1, 5, 7, 2, 0, 4, 5, 7, 1, 4, 3, 6, 7, 5, 6, 3, 1, 6, 3, 6, 0, 4, 6, 2, 5, 1, 2, 2, 7, 0, 5, 4, 4, 1, 1, 1, 4, 5, 7, 0, 2, 3, 6, 3, 3, 2, 3]
Offspring 2 Representation: [7, 1, 2, 4, 5, 0, 4, 3, 0, 7, 6, 4, 2, 2, 7, 7, 3, 5, 5, 5, 6, 1, 1, 6, 2, 7, 1, 4, 0, 0, 1, 4, 0, 5, 4, 6, 2, 6, 0, 7, 3, 6, 7, 2, 4, 1, 6, 3, 4, 1, 7, 0, 2, 6, 5, 0, 3, 5, 3, 2, 3, 3, 1, 5]


## Partially Matched Crossover

In [15]:
def partially_matched_crossover(parent1_repr, parent2_repr):

    '''
    
    Definition
        In Partially Matched Crossover, two random indices are selected to create a 'window' in
        each parent solution. Values within the window are copied to the opposite offspring (i.e. 
        Parent 1 copied into Offspring 2). Remaining indices are filled with values from the other
        parent, skipping values already present in offspring from window swap.
    
    Parameters
        parent1_repr: Parent 1 representataion (list)
        parent2_repr: Parent 2 representation (list)
    
    Returns
        Pair of offspring with same type as parents
        
    '''

    tagged_p1 = tag_solution(parent1_repr)
    tagged_p2 = tag_solution(parent2_repr)

    size = len(tagged_p1)
    # sanity check
    print('Size of tagged parents:', size)

    # randomly choose two indices for crossover window
    first_idx = random.randint(0, len(tagged_p1)-1)
    second_idx = first_idx
    
    # We want to get two indexes that are at least 2 genes away
    while abs(second_idx-first_idx) <= 1:
        second_idx = random.randint(0, len(tagged_p1)-1)

    # Ensure first_idx < second_idx
    if first_idx > second_idx:
        first_idx, second_idx = second_idx, first_idx

    # viewing indices
    print('First index:', first_idx)
    print('Second index:', second_idx)

    # initiating empty offspring of correct size
    tagged_offspring1 = [None] * size
    tagged_offspring2 = [None] * size

    for i in range(first_idx, second_idx +1):

        # take values from window of opposite parent
        tagged_offspring1[i] = tagged_p2[i]
        tagged_offspring2[i] = tagged_p1[i]

    # to fill the leftover positions
    def fill_remaining(tagged_parent, tagged_offspring):
        for i in range(size):

            # leave the indices in the window alone
            if i >= first_idx and i <= second_idx:
                continue

            # copy any remaining values from primary parent
            for value in tagged_parent:
                if value not in tagged_offspring:
                    tagged_offspring[i] = value
                    break

    fill_remaining(tagged_p1, tagged_offspring1)
    fill_remaining(tagged_p2, tagged_offspring2)

    # sanity check
    print('Tagged Offspring 1:', tagged_offspring1)
    print('Tagged Offspring 2:', tagged_offspring2)

    offspring1_repr = untag_solution(tagged_offspring1)
    offsrping2_repr = untag_solution(tagged_offspring2)

    return offspring1_repr, offsrping2_repr

In [16]:
# testing partially matched crossover with toy values
parent1 = [1, 2, 1, 3, 2, 3]
parent2 = [3, 2, 1, 1, 3, 2]

print('Parent 1:', parent1)
print('Parent 2:', parent2)

print('Tagged parent 1:', tag_solution(solution_repr=parent1))
print('Tagged parent 2:', tag_solution(solution_repr=parent2))

offspring1, offspring2 = partially_matched_crossover(parent1_repr=parent1, parent2_repr=parent2)

print('Offspring 1:', offspring1)
print('Offspring 2:', offspring2)

Parent 1: [1, 2, 1, 3, 2, 3]
Parent 2: [3, 2, 1, 1, 3, 2]
Tagged parent 1: [(1, 0), (2, 0), (1, 1), (3, 0), (2, 1), (3, 1)]
Tagged parent 2: [(3, 0), (2, 0), (1, 0), (1, 1), (3, 1), (2, 1)]
Size of tagged parents: 6
First index: 0
Second index: 4
Tagged Offspring 1: [(3, 0), (2, 0), (1, 0), (1, 1), (3, 1), (2, 1)]
Tagged Offspring 2: [(1, 0), (2, 0), (1, 1), (3, 0), (2, 1), (3, 1)]
Offspring 1: [3, 2, 1, 1, 3, 2]
Offspring 2: [1, 2, 1, 3, 2, 3]


In [17]:
# testing on a real solution
parent1 = WSOSolution().repr
parent2 = WSOSolution().repr

print('Parent 1:', parent1)
print('Parent 2:', parent2)

offspring1_repr, offsrping2_repr = partially_matched_crossover(parent1_repr=parent1, parent2_repr=parent2)

print('Offspring 1 Representation:', offspring1_repr)
print('Offspring 2 Representation:', offsrping2_repr)

Parent 1: [7, 4, 0, 4, 5, 2, 1, 5, 2, 7, 0, 6, 5, 4, 3, 1, 7, 3, 4, 2, 2, 2, 1, 3, 7, 0, 2, 7, 1, 6, 3, 1, 0, 6, 5, 5, 2, 4, 5, 0, 4, 0, 3, 6, 7, 5, 0, 1, 1, 7, 7, 6, 2, 4, 6, 4, 3, 1, 5, 6, 3, 0, 6, 3]
Parent 2: [1, 5, 5, 7, 3, 2, 1, 7, 5, 4, 4, 1, 7, 4, 2, 0, 1, 5, 3, 3, 5, 7, 3, 4, 4, 1, 6, 0, 2, 0, 6, 2, 5, 6, 7, 5, 6, 4, 6, 0, 3, 0, 6, 2, 2, 7, 3, 3, 6, 6, 1, 0, 2, 1, 1, 7, 7, 2, 4, 0, 4, 3, 0, 5]
Size of tagged parents: 64
First index: 9
Second index: 27
Tagged Offspring 1: [(7, 0), (5, 0), (2, 0), (1, 0), (5, 1), (7, 1), (5, 2), (3, 0), (1, 1), (4, 0), (4, 1), (1, 2), (7, 2), (4, 2), (2, 1), (0, 0), (1, 3), (5, 3), (3, 1), (3, 2), (5, 4), (7, 3), (3, 3), (4, 3), (4, 4), (1, 4), (6, 0), (0, 1), (2, 2), (2, 3), (2, 4), (0, 2), (2, 5), (7, 4), (6, 1), (0, 3), (6, 2), (2, 6), (5, 5), (0, 4), (4, 5), (0, 5), (3, 4), (6, 3), (7, 5), (5, 6), (0, 6), (1, 5), (1, 6), (7, 6), (7, 7), (6, 4), (2, 7), (4, 6), (6, 5), (4, 7), (3, 5), (1, 7), (5, 7), (6, 6), (3, 6), (0, 7), (6, 7), (3, 7)]
Ta

## Position Based Crossover

In [18]:
def pos_based_crossover(parent1_repr, parent2_repr):

    '''
    Definition
        This operator is very similar to martially matched crossover - the difference being random 
        indices are exchanged, rather than a window. Position based crossover selects a random set 
        of indices from the parent solutions, then copies the values from these indices in each 
        parent to one of the offspring. The following indices are filled with values from the opposite
        parent, in the order in which they appear.
    
    Parameters:
        parent1_repr: Parent 1 representataion (list)
        parent2_repr: Parent 2 representation (list)

    Returns:
        Pair of offspring with same type as parents
    '''

    tagged_p1 = tag_solution(parent1_repr)
    tagged_p2 = tag_solution(parent2_repr)

    size = len(tagged_p1)

    # choose random selection of indices
    num_indices = random.randint(1, size)
    selected_indices = sorted(random.sample(range(size), num_indices))

    print('Number of indices selected:', num_indices)
    print('Randomly selected indices:', selected_indices)

    # initializing empty offspring of correct size
    tagged_offspring1 = [None] * size
    tagged_offspring2 = [None] * size

    for i in selected_indices:
        tagged_offspring1[i] = tagged_p1[i]
        tagged_offspring2[i] = tagged_p2[i]

    # fill remaining indices with values from opposite parent, in the order which they appear, skipping duplicates
    def fill_remaining(tagged_parent, tagged_offspring):
        for i in range(size):

            # leave the indices that were randomly selected alone
            if i in selected_indices:
                continue

            # copy any remaining values from opposite parent
            for value in tagged_parent:
                if value not in tagged_offspring:
                    tagged_offspring[i] = value
                    break

    fill_remaining(tagged_p1, tagged_offspring2)
    fill_remaining(tagged_p2, tagged_offspring1)

    # sanity check
    print('Tagged Offspring 1:', tagged_offspring1)
    print('Tagged Offspring 2:', tagged_offspring2)

    offspring1_repr = untag_solution(tagged_offspring1)
    offsrping2_repr = untag_solution(tagged_offspring2)

    return offspring1_repr, offsrping2_repr

In [19]:
# testing partially matched crossover with toy values
parent1 = [1, 2, 1, 3, 2, 3]
parent2 = [3, 2, 1, 1, 3, 2]

print('Parent 1:', parent1)
print('Parent 2:', parent2)

print('Tagged parent 1:', tag_solution(solution_repr=parent1))
print('Tagged parent 2:', tag_solution(solution_repr=parent2))

offspring1, offspring2 = pos_based_crossover(parent1_repr=parent1, parent2_repr=parent2)

print('Offspring 1:', offspring1)
print('Offspring 2:', offspring2)

Parent 1: [1, 2, 1, 3, 2, 3]
Parent 2: [3, 2, 1, 1, 3, 2]
Tagged parent 1: [(1, 0), (2, 0), (1, 1), (3, 0), (2, 1), (3, 1)]
Tagged parent 2: [(3, 0), (2, 0), (1, 0), (1, 1), (3, 1), (2, 1)]
Number of indices selected: 4
Randomly selected indices: [0, 1, 2, 4]
Tagged Offspring 1: [(1, 0), (2, 0), (1, 1), (3, 0), (2, 1), (3, 1)]
Tagged Offspring 2: [(3, 0), (2, 0), (1, 0), (1, 1), (3, 1), (2, 1)]
Offspring 1: [1, 2, 1, 3, 2, 3]
Offspring 2: [3, 2, 1, 1, 3, 2]


In [20]:
# testing on a real solution
parent1 = WSOSolution().repr
parent2 = WSOSolution().repr

print('Parent 1:', parent1)
print('Parent 2:', parent2)

offspring1_repr, offsrping2_repr = pos_based_crossover(parent1_repr=parent1, parent2_repr=parent2)

print('Offspring 1 Representation:', offspring1_repr)
print('Offspring 2 Representation:', offsrping2_repr)

Parent 1: [4, 0, 4, 5, 2, 7, 7, 5, 7, 0, 6, 4, 6, 6, 2, 6, 4, 0, 3, 1, 6, 6, 5, 1, 5, 3, 7, 4, 4, 3, 2, 4, 2, 2, 3, 1, 3, 5, 1, 7, 0, 3, 2, 6, 0, 7, 2, 3, 6, 0, 7, 1, 1, 3, 5, 5, 0, 7, 4, 1, 2, 0, 1, 5]
Parent 2: [6, 6, 2, 5, 1, 0, 5, 5, 5, 7, 3, 7, 7, 3, 3, 6, 1, 3, 2, 7, 0, 7, 4, 0, 0, 2, 3, 4, 2, 5, 0, 5, 4, 0, 6, 1, 2, 4, 7, 6, 2, 3, 7, 4, 4, 3, 0, 1, 4, 4, 7, 1, 6, 1, 6, 6, 5, 3, 1, 1, 2, 5, 2, 0]
Number of indices selected: 12
Randomly selected indices: [5, 7, 8, 10, 13, 16, 19, 25, 28, 33, 57, 63]
Tagged Offspring 1: [(6, 1), (2, 0), (5, 0), (0, 0), (5, 2), (7, 0), (5, 3), (5, 1), (7, 2), (3, 0), (6, 0), (7, 1), (3, 2), (6, 2), (1, 1), (3, 3), (4, 3), (2, 1), (7, 3), (1, 0), (0, 1), (7, 4), (4, 0), (0, 2), (0, 3), (3, 1), (2, 2), (3, 4), (4, 5), (4, 1), (2, 3), (5, 4), (0, 4), (2, 4), (5, 5), (4, 2), (0, 5), (6, 3), (1, 2), (7, 5), (6, 4), (2, 5), (3, 5), (7, 6), (4, 4), (3, 6), (0, 6), (1, 3), (4, 6), (4, 7), (1, 4), (6, 5), (1, 5), (6, 6), (6, 7), (5, 6), (3, 7), (7, 7), (1, 6

# MUTATION OPERATORS

## Block Swap Mutation

In [18]:
def block_swap_mutation(solution_repr, block_size = 4, mut_prob = 0.2):

    '''
    Definition
        Appplies a modified swap mutation (swapping two blocks of indices) in a list represetation of a solution.
        Applied with given mutation probability.

        Block swap randomly selects two segments of the solution represenation
        (segement of length block_size) and swaps all values in the two blocks.
    
    Parameters
        solution_repr: Representation of solution to mutate (list)
        block_size: length of segments to be swapped
        mut_prob: probability of performing the mutation

    Returns
        Copy of original solution if mutation probability condition is not met
        New mutated solution if mutation probability is met

    '''

    new_repr = deepcopy(solution_repr)

    if random.random() <= mut_prob:
    
        # randomly choose two indices for block start positions - must be block_size genes before end
        first_idx = random.randint(0, len(solution_repr) - 1 - block_size)
        second_idx = first_idx

        # We want to get two indexes that are at least block_size number of genes away from one another
        while abs(second_idx-first_idx) <= block_size:
            second_idx = random.randint(0, len(solution_repr) - 1 - block_size)
        

        # Ensure first_idx < second_idx
        if first_idx > second_idx:
            first_idx, second_idx = second_idx, first_idx

        # viewing indices
        print('First index:', first_idx)
        print('Second index:', second_idx)

        for i in range(block_size):
            solution_repr[first_idx + i], solution_repr[second_idx + i] = solution_repr[second_idx + i], solution_repr[first_idx + i]

        return solution_repr
    
    else:
        return new_repr


In [19]:
# testing block mutation with toy values
parent1 = [1, 2, 2, 3, 1, 3, 2, 3, 1, ]

print('Parent 1:', parent1)

offspring = block_swap_mutation(parent1, 2, mut_prob=1)

print('Mutated Offspring:', offspring)

Parent 1: [1, 2, 2, 3, 1, 3, 2, 3, 1]
First index: 2
Second index: 6
Mutated Offspring: [1, 2, 2, 3, 1, 3, 2, 3, 1]


In [26]:
# testing block mutation on real solution

parent_repr = WSOSolution().repr

print('Prarent Respresentation:', parent_repr)

offspring = block_swap_mutation(parent_repr, 6, mut_prob=1)

print('Mutated Offspring:', offspring)


Prarent Respresentation: [1, 3, 4, 7, 0, 2, 7, 4, 3, 6, 3, 4, 5, 5, 6, 2, 0, 1, 0, 5, 2, 4, 6, 2, 4, 0, 7, 1, 5, 3, 3, 2, 0, 2, 4, 7, 0, 1, 6, 2, 6, 0, 2, 5, 1, 4, 5, 3, 7, 1, 7, 4, 1, 5, 6, 3, 6, 6, 7, 5, 0, 3, 1, 7]
First index: 42
Second index: 56
Mutated Offspring: [1, 3, 4, 7, 0, 2, 7, 4, 3, 6, 3, 4, 5, 5, 6, 2, 0, 1, 0, 5, 2, 4, 6, 2, 4, 0, 7, 1, 5, 3, 3, 2, 0, 2, 4, 7, 0, 1, 6, 2, 6, 0, 6, 6, 7, 5, 0, 3, 7, 1, 7, 4, 1, 5, 6, 3, 2, 5, 1, 4, 5, 3, 1, 7]


## Shuffle Subsequence Mutation

In [25]:
def shuffle_subsequence_mutation(solution_repr, mut_prob = 0.2):

    '''
    Definition
        Shuffle Subsequence is a modification of Inverse Subsequence Mutation presented in
        practiacl classes. It applies a shuffling of values between two randomly selected indices.
        Applied with given mutation probability.

        Shuffle Subsequence mutation selects two random indices from the solution,
        and randomly shuffles the values between them.

    Parameters
        solution_repr: Representation of solution to mutate (list)
        mut_prob: probability of performing the mutation
    
    Returns
        Copy of original solution if mutation probability condition is not met
        New mutated solution if mutation probability is met
    
    '''

    new_repr = deepcopy(solution_repr)

    if random.random() <= mut_prob:

        # Select two distinct indices
        first_idx = random.randint(0, len(new_repr)-1)
        second_idx = first_idx

        # We want to get two indices that are at least 2 genes away
        while abs(second_idx-first_idx) <= 1:
            second_idx = random.randint(0, len(new_repr)-1)
    
        # Ensure first_idx < second_idx
        if first_idx > second_idx:
            first_idx, second_idx = second_idx, first_idx

        print('First index:', first_idx)
        print('Second index:', second_idx)

        # Reverse between first and second index
        subsequence = new_repr[first_idx:second_idx]

        # Shuffle the subsequence in place
        random.shuffle(subsequence)
        
        # Reassign shuffled values between 2 indices
        new_repr[first_idx:second_idx] = subsequence

        return new_repr
    
    else:
        return new_repr

In [27]:
# testing shuffle subsequence mutation with toy values
parent1 = [1, 2, 2, 3, 1, 3, 2, 3, 1, ]

print('Parent 1:', parent1)

offspring = shuffle_subsequence_mutation(parent1, mut_prob=1)

print('Mutated Offspring:', offspring)

Parent 1: [1, 2, 2, 3, 1, 3, 2, 3, 1]
First index: 1
Second index: 4
Mutated Offspring: [1, 2, 2, 3, 1, 3, 2, 3, 1]


In [28]:
# testing shuffle subsquence mutation with real solutions

parent_repr = WSOSolution().repr

print('Prarent Respresentation:', parent_repr)

offspring = shuffle_subsequence_mutation(parent_repr, mut_prob=1)

print('Mutated Offspring:', offspring)

Prarent Respresentation: [5, 7, 2, 0, 6, 3, 2, 1, 7, 5, 5, 7, 4, 6, 0, 1, 2, 3, 2, 6, 6, 3, 4, 2, 5, 3, 6, 1, 7, 1, 1, 3, 4, 1, 4, 4, 5, 2, 5, 4, 0, 7, 4, 2, 7, 1, 0, 3, 6, 0, 7, 5, 5, 1, 2, 4, 6, 6, 3, 0, 0, 3, 7, 0]
First index: 4
Second index: 6
Mutated Offspring: [5, 7, 2, 0, 6, 3, 2, 1, 7, 5, 5, 7, 4, 6, 0, 1, 2, 3, 2, 6, 6, 3, 4, 2, 5, 3, 6, 1, 7, 1, 1, 3, 4, 1, 4, 4, 5, 2, 5, 4, 0, 7, 4, 2, 7, 1, 0, 3, 6, 0, 7, 5, 5, 1, 2, 4, 6, 6, 3, 0, 0, 3, 7, 0]


## N-Swap Mutation

In [29]:
def n_swap_mutation(solution_repr, mut_prob = 0.2, n = 8):

    '''
    Defition
        N-Swap Mutation is a modification of Swap Mutation present in practical classes. It
        selects n random pairs of iniduces to swap perform swap mutation.
        Applied with given mutation probability.

        N-Swap mutation randomly selects n pairs of positions from given solution,
        and swaps their values.

    Parameters
        solution_repr: Representation of solution to mutate (list)
        mut_prob: probability of performing the mutation
        n: number of pairs of indices to swap

    Returns:
        Copy of original solution if mutation probability condition is not met
        New mutated solution if mutation probability is met
        
    '''

    new_repr = deepcopy(solution_repr)

    if random.random() <= mut_prob:
        for _ in range (n):
            i, j = random.sample(range(len(solution_repr)), 2)
            new_repr[i], new_repr[j] = new_repr[j], new_repr[i]

            print('Index 1:', i)
            print('Index 2:', j)

        return new_repr
    
    else:
        return new_repr

In [30]:
# testing n-swap mutation with toy values
parent1 = [1, 2, 2, 3, 1, 3, 2, 3, 1, ]

print('Parent 1:', parent1)

offspring = n_swap_mutation(parent1, mut_prob=1, n=1)

print('Mutated Offspring:', offspring)

Parent 1: [1, 2, 2, 3, 1, 3, 2, 3, 1]
Index 1: 6
Index 2: 0
Mutated Offspring: [2, 2, 2, 3, 1, 3, 1, 3, 1]


In [31]:
# testing n-swap mutation with real solutions

parent_repr = WSOSolution().repr

print('Prarent Respresentation:', parent_repr)

offspring = n_swap_mutation(parent_repr, n=4, mut_prob=1)

print('Mutated Offspring:', offspring)

Prarent Respresentation: [2, 1, 6, 5, 6, 3, 4, 0, 3, 3, 6, 2, 4, 6, 7, 2, 0, 4, 2, 0, 5, 5, 0, 7, 1, 3, 1, 0, 5, 3, 5, 4, 4, 5, 4, 1, 6, 4, 6, 1, 7, 7, 3, 7, 7, 0, 5, 7, 0, 3, 4, 6, 2, 1, 1, 3, 6, 2, 5, 2, 0, 2, 7, 1]
Index 1: 21
Index 2: 13
Index 1: 61
Index 2: 62
Index 1: 35
Index 2: 33
Index 1: 51
Index 2: 16
Mutated Offspring: [2, 1, 6, 5, 6, 3, 4, 0, 3, 3, 6, 2, 4, 5, 7, 2, 6, 4, 2, 0, 5, 6, 0, 7, 1, 3, 1, 0, 5, 3, 5, 4, 4, 1, 4, 5, 6, 4, 6, 1, 7, 7, 3, 7, 7, 0, 5, 7, 0, 3, 4, 0, 2, 1, 1, 3, 6, 2, 5, 2, 0, 7, 2, 1]


## Displacement Mutation

Potential Adjustments:
- Could get a modification that prevents the cut subsequence from being re-inserted into the same position

In [32]:
def displacement_mutation(solution_repr, mut_prob = 0.2):

    '''
    Definition
        Displacement Mutation selects a random subsequence from the solution
        representation. This subsequence is removed, then re-inserted into the solution
        at a random position.
    
    Parameters
        solution_repr: Representation of solution to mutate (list)
        mut_prob: probability of performing the mutation

    Returns
        Copy of original solution if mutation probability condition is not met
        New mutated solution if mutation probability is met
    '''
    
    new_repr = deepcopy(solution_repr)

    if random.random() <= mut_prob:

        # Select two distinct indices
        first_idx = random.randint(0, len(new_repr)-1)
        second_idx = first_idx

        # We want to get two indices that are at least 2 genes away
        while abs(second_idx-first_idx) <= 1:
            second_idx = random.randint(0, len(new_repr)-1)
    
        # Ensure first_idx < second_idx
        if first_idx > second_idx:
            first_idx, second_idx = second_idx, first_idx

        print('First index:', first_idx)
        print('Second index:', second_idx)

        subsequence = new_repr[first_idx:second_idx+1]

        # remove selected subsequence from solution representation
        remaining = new_repr[: first_idx] + new_repr[second_idx + 1:]

        # choose a random position to re-insert the cut segment
        insert_position = random.randint(0, len(remaining))
        print('Position where subsequence is inserted:', insert_position)

        new_repr = remaining[: insert_position] + subsequence + remaining[insert_position: ]

        return new_repr
    
    else:
        return new_repr

In [33]:
# testing displacement mutation with toy values
parent1 = [1, 2, 2, 3, 1, 3, 2, 3, 1]

print('Parent 1:', parent1)

offspring = displacement_mutation(parent1, mut_prob=1)

print('Mutated Offspring:', offspring)

Parent 1: [1, 2, 2, 3, 1, 3, 2, 3, 1]
First index: 3
Second index: 5
Position where subsequence is inserted: 6
Mutated Offspring: [1, 2, 2, 2, 3, 1, 3, 1, 3]


In [34]:
# testing n-swap mutation with real solutions

parent_repr = WSOSolution().repr

print('Prarent Respresentation:', parent_repr)

offspring = displacement_mutation(parent_repr, mut_prob=1)

print('Mutated Offspring:', offspring)

Prarent Respresentation: [7, 7, 0, 2, 0, 5, 4, 6, 2, 3, 5, 0, 5, 4, 1, 6, 6, 6, 7, 1, 6, 4, 5, 0, 3, 3, 0, 3, 2, 2, 4, 2, 1, 2, 3, 5, 6, 7, 7, 3, 5, 1, 4, 3, 1, 6, 0, 6, 7, 0, 2, 3, 7, 4, 1, 4, 4, 0, 2, 5, 1, 5, 1, 7]
First index: 5
Second index: 19
Position where subsequence is inserted: 21
Mutated Offspring: [7, 7, 0, 2, 0, 6, 4, 5, 0, 3, 3, 0, 3, 2, 2, 4, 2, 1, 2, 3, 5, 5, 4, 6, 2, 3, 5, 0, 5, 4, 1, 6, 6, 6, 7, 1, 6, 7, 7, 3, 5, 1, 4, 3, 1, 6, 0, 6, 7, 0, 2, 3, 7, 4, 1, 4, 4, 0, 2, 5, 1, 5, 1, 7]
